# Making a map (choropleth or geo scatter) for job postings

In [1]:
# plotting pacakges
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
%matplotlib inline

# assortment of other libraries
import re
import string
from collections import Counter
import pandas as pd
import numpy as np
import arrow
from geopy import geocoders
import geocoder
import threading
import time


import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode(connected=True)

In [166]:
test = pd.DataFrame({"something":[1,2,3,np.nan], "else":[3,4,5,4]})

test.loc[:,['something','else']]

,something,else
0,1.0,3
1,2.0,4
2,3.0,5
3,NaN,4


In [176]:
import string
test_word = "hi,"
table = str.maketrans({key: None for key in string.punctuation})
new_s = test_word.translate(table)   
new_s

'hi'

In [142]:
test1 = pd.read_csv("../data/raw_scrapes/glassdoor-df-06-06-2018.csv", encoding="ISO-8859-1")
test1_processed = pd.read_csv("../data/scrapes/cleaned_scrapes/glassdoor-df-06-06-2018.csv", encoding="ISO-8859-1")
test2 = pd.read_csv("../data/scrapes/cleaned_filtered_scrapes/glassdoor-df-06-22-2018.csv", encoding="ISO-8859-1")

In [167]:
test_old = pd.read_csv("../data/cleaned_ranked_scrapes/glassdoor-df-06-06-2018.csv", encoding="ISO-8859-1")
test_new = pd.read_csv("../data/raw_scrapes/glassdoor-df-06-08-2018.csv", encoding="ISO-8859-1")

In [168]:
print(test_old.shape)
print(test_new.shape)

(4906, 11)
(5608, 5)


In [170]:
test = test_new.merge(test_old, on=['company','position','location','description'], how="outer", indicator=True)

In [172]:
test.head(n=100)

,position,company,location,description,link_x,link_y,days_posted,date_posted,applied,date_applied,strikes,rank,_merge
0,Data Scientist - New Grad,"Quora, Inc.","Mountain View, CA","Because Quora is such a data-driven company, o...",https://www.glassdoor.com/partner/jobListing.h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Data Scientist - New Grad,"Quora, Inc.","Mountain View, CA","Because Quora is such a data-driven company, o...",https://www.glassdoor.com/partner/jobListing.h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,Data Scientist - New Grad,Yelp,"San Francisco, CA","At Yelp, we see our 148 million reviews not on...",https://www.glassdoor.com/partner/jobListing.h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,Data Scientist,Expedia,"Bellevue, WA",ExpediaWe are looking for a data scientist to ...,https://www.glassdoor.com/partner/jobListing.h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Data Scientist,Expedia,"Bellevue, WA",ExpediaWe are looking for a data scientist to ...,https://www.glassdoor.com/partner/jobListing.h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5,New Grad - Data Scientist,Viasat,"Boston, MA",Job ResponsibilitiesBecome a personal contribu...,https://www.glassdoor.com/partner/jobListing.h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6,New Grad - Research Scientist (PhD),Visa,"Palo Alto, CA","Company DescriptionCommon Purpose, Uncommon Op...",https://www.glassdoor.com/partner/jobListing.h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7,Machine Learning Scientist (University Grad),Electronic Arts,"Redwood City, CA",We are EA\r\n\r\nAnd we make games how cool is...,https://www.glassdoor.com/partner/jobListing.h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
8,Machine Learning Scientist (University Grad),Capital Games,"Redwood City, CA",Electronic Arts Inc. is a leading global inter...,https://www.glassdoor.com/partner/jobListing.h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9,"Quantitative Researcher, Growth - University Grad",Facebook,"Menlo Park, CA",Facebook's mission is to give people the power...,https://www.glassdoor.com/partner/jobListing.h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [155]:
test_old.head(n=4)

,position,company,location,description,link,days_posted,date_posted,applied,date_applied,strikes,rank
0,Data Scientist - New Grad,Yelp,"San Francisco, CA","At Yelp, we see our 148 million reviews not on...",https://www.glassdoor.com/partner/jobListing.h...,0.0,06-11-2018,No,NaN,2.0,22.0
1,Data Scientist - New Grad,"Quora, Inc.","Mountain View, CA","Because Quora is such a data-driven company, o...",https://www.glassdoor.com/partner/jobListing.h...,0.0,06-11-2018,No,NaN,2.0,22.0
2,Data Scientist,Expedia,"Bellevue, WA",ExpediaWe are looking for a data scientist to ...,https://www.glassdoor.com/partner/jobListing.h...,0.0,06-11-2018,No,NaN,2.0,22.0
3,Data Scientist,3Q Digital,"New York, NY",3Q Digital is Silicon Valley's agency of recor...,https://www.glassdoor.com/partner/jobListing.h...,0.0,06-11-2018,No,NaN,2.0,22.0


In [156]:
test_new.head(n=4)

,position,company,location,description,link
0,Data Scientist - New Grad,"Quora, Inc.","Mountain View, CA","Because Quora is such a data-driven company, o...",https://www.glassdoor.com/partner/jobListing.h...
1,Data Scientist - New Grad,Yelp,"San Francisco, CA","At Yelp, we see our 148 million reviews not on...",https://www.glassdoor.com/partner/jobListing.h...
2,New Grad - Data Scientist,Viasat,"Boston, MA",Job ResponsibilitiesBecome a personal contribu...,https://www.glassdoor.com/partner/jobListing.h...
3,Data Scientist,Expedia,"Bellevue, WA",ExpediaWe are looking for a data scientist to ...,https://www.glassdoor.com/partner/jobListing.h...


***
***

In [143]:
test1.shape

(5809, 5)

In [144]:
test1_processed.shape

(4906, 11)

In [145]:
test1_processed.columns

Index(['position', 'company', 'location', 'description', 'link', 'days_posted',
       'date_posted', 'applied', 'date_applied', 'strikes', 'rank'],
      dtype='object')

In [141]:
test2.shape

(3986, 11)

In [136]:
test

,else,something
3,no,3
1,no,2
0,yes,3
2,yes,1


***

### Threaded version of getting coordinates

only problem is it just gives me none's for most of the coordinates... not worth troubleshooting right now b/c even though it's much faster than non-threaded... non-threaded only takes about 30 mins.

In [ ]:
%%time

# returns [location, latitude, longitude]
class Coordinate():

    def __init__(self, locations):
        
        self.locations = locations
        self.location_threads = []
        self.coordinates = []
        self.lock = threading.Lock()
        
    def start_threads(self):
        
        section = len(self.locations)//40
        
        for i in range(1,41):
            self.location_threads.append(threading.Thread(target=self.get_coordinates, 
                                                          args=(i, ((i-1)*section),(i*section))))
            self.location_threads[i-1].start()
        self.location_threads.append(threading.Thread(target=self.get_coordinates,
                                                      args=(i, 40*section,'end')))
        self.location_threads[40].start()

    def get_coordinates(self, i, start, stop):
        gn = geocoders.GeoNames(username='kiwirider')
        section = len(self.locations)//40

        tmp = []
        
        if stop == 'end':
            locations = self.locations[start:]
        else:
            locations = self.locations[start:stop]
            
        for j,location in enumerate(locations):
            
            if (j > 0) and (j % 30 == 0):
                print("Worker {0}: currently {1}% finished.".format(i, round((j/section)*100),0))
            
            if location == 'none':
                tmp.append(['none','none','none'])
                continue
            try:
                loc = gn.geocode(location)
                tmp.append([location, loc.latitude, loc.longitude])
            except:
                tmp.append([location,'none','none'])
                
        with self.lock:
            self.coordinates += tmp
            
            
if __name__ == '__main__':
    
    coords = Coordinate(locations)
    coords.start_threads()
    for thread in coords.location_threads:
        thread.join()
    print("Finished getting cooridnates!")

***

### Non-threaded version of getting coordinates from city locations [WORKING!!]

In [ ]:
%%time

df = pd.read_csv('../data/scrapes/clean_filtered_scrapes/glassdoor-df-06-12-2018.csv', encoding="ISO-8859-1")
locations = df['location'].tolist()[:100]

def get_coordinates(locations):
    
    tmp = []

    for i,location in enumerate(locations):
        
        if i % 50 == 0:
            gn = geocoders.GeoNames(username='kiwirider')

        if location == 'none':
            tmp.append(['none','none','none'])
            continue
        try:
            loc = gn.geocode(location)
            tmp.append([location, loc.latitude, loc.longitude])
        except:
            tmp.append([location,'none','none'])

    return tmp

coords = get_coordinates(locations)
clean_coords = [x for x in coords if 'none' not in x]

loc = [x[0] for x in clean_coords]
lat = [x[1] for x in clean_coords]
lon = [x[2] for x in clean_coords]

combined_loc = []

for i in range(71):
    combined_loc.append([loc[i],lat[i],lon[i]])

loc_new = []
lat_new = []
lon_new = []
count = 0
for i in range(len(combined_loc)):
    try:
        state = loc[i].split(",")[1].strip()
        if state=='CA':
            loc_new.append(combined_loc[i][0])
            lat_new.append(combined_loc[i][1])
            lon_new.append(combined_loc[i][2])
    except:
        count += 1
print(count)   

***

### US Choropleth

In [106]:
rgb = []

for i in range(len(scl)):
    if(len(rgb) < 20):
        h = scl[i].lstrip('#')
        rgb.append([i/10,'rgb{}'.format(tuple(int(h[i:i+2], 16) for i in (0, 2 ,4)))])
        rgb.append([(i+1)/10,'rgb{}'.format(tuple(int(h[i:i+2], 16) for i in (0, 2 ,4)))])

In [107]:
rgb

[[0.0, 'rgb(236, 239, 241)'],
 [0.1, 'rgb(236, 239, 241)'],
 [0.1, 'rgb(218, 223, 228)'],
 [0.2, 'rgb(218, 223, 228)'],
 [0.2, 'rgb(200, 208, 215)'],
 [0.3, 'rgb(200, 208, 215)'],
 [0.3, 'rgb(182, 192, 201)'],
 [0.4, 'rgb(182, 192, 201)'],
 [0.4, 'rgb(164, 176, 188)'],
 [0.5, 'rgb(164, 176, 188)'],
 [0.5, 'rgb(145, 161, 175)'],
 [0.6, 'rgb(145, 161, 175)'],
 [0.6, 'rgb(127, 145, 162)'],
 [0.7, 'rgb(127, 145, 162)'],
 [0.7, 'rgb(109, 129, 148)'],
 [0.8, 'rgb(109, 129, 148)'],
 [0.8, 'rgb(91, 114, 135)'],
 [0.9, 'rgb(91, 114, 135)'],
 [0.9, 'rgb(73, 98, 122)'],
 [1.0, 'rgb(73, 98, 122)']]

In [115]:
df = pd.read_csv('../data/scrapes/clean_filtered_scrapes/glassdoor-df-06-12-2018.csv', encoding="ISO-8859-1")
locations = df['location'].tolist()

locs = []
count = 0
for i in range(len(locations)):
    try:
        state = locations[i].split(",")[1].strip()
        if len(state)==2:
            locs.append(state)
    except:
        count += 1
locs = Counter(locs)

# scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
#             [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

# colors = np.linspace(50,220,10)[::-1]

# colors = []
# scl = []

# for i,color in enumerate(colors):
#     scl.append([i/10,'rgb({0},{0},{0})'.format(color)])
#     scl.append([(i+1)/10,'rgb({0},{0},{0})'.format(color)])


   

scl=['#ECEFF1','#DADFE4','#C8D0D7','#B6C0C9','#A4B0BC','#91A1AF','#7F91A2','#6D8194','#5B7287','#49627A','#37536D',
     '#324C64','#2E445A','#293D50','#233546','#1E2E3C','#192632','#141F28','#0F171E','#0A1014']

rgb = []

for index in range(len(scl)):
    if(len(rgb) < 20):
        h = scl[index+3].lstrip('#')
        rgb.append([index/10,'rgb{}'.format(tuple(int(h[i:i+2], 16) for i in (0, 2 ,4)))])
        rgb.append([(index+1)/10,'rgb{}'.format(tuple(int(h[i:i+2], 16) for i in (0, 2 ,4)))])
print(rgb)    

data = [dict(type='choropleth', colorscale=rgb, 
             autocolorscale=False, locationmode='USA-states', locations=list(locs.keys()), z=list(locs.values()),
             marker = dict(line=dict(color='rgb(255,255,255)', width=2)),
             colorbar=dict(title="Number of Listings"),
             zauto=False,
             zmin=0,
             zmax=150)]
layout = dict(title="Job Listings by State",
              geo = dict(scope='usa'))

fig = dict(data=data, layout=layout)
offline.iplot(fig)

[[0.0, 'rgb(182, 192, 201)'], [0.1, 'rgb(182, 192, 201)'], [0.1, 'rgb(164, 176, 188)'], [0.2, 'rgb(164, 176, 188)'], [0.2, 'rgb(145, 161, 175)'], [0.3, 'rgb(145, 161, 175)'], [0.3, 'rgb(127, 145, 162)'], [0.4, 'rgb(127, 145, 162)'], [0.4, 'rgb(109, 129, 148)'], [0.5, 'rgb(109, 129, 148)'], [0.5, 'rgb(91, 114, 135)'], [0.6, 'rgb(91, 114, 135)'], [0.6, 'rgb(73, 98, 122)'], [0.7, 'rgb(73, 98, 122)'], [0.7, 'rgb(55, 83, 109)'], [0.8, 'rgb(55, 83, 109)'], [0.8, 'rgb(50, 76, 100)'], [0.9, 'rgb(50, 76, 100)'], [0.9, 'rgb(46, 68, 90)'], [1.0, 'rgb(46, 68, 90)']]


***

### Choropleth by state counties

In [20]:
import plotly.plotly as py
import plotly.figure_factory as ff

import numpy as np
import pandas as pd

df = pd.read_csv('../data/scrapes/clean_filtered_scrapes/glassdoor-df-06-12-2018.csv', encoding="ISO-8859-1")
locations = df['location'].tolist()[:100]

df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/minoritymajority.csv')
df_sample_r = df_sample[df_sample['STNAME'] == 'California']
cali = df_sample_r

locs = []
count = 0
for i in range(len(locations)):
    try:
        state = locations[i].split(",")[1].strip()
        if state=='CA':
            locs.append(locations[i])
    except:
        count += 1


counties = []

for location in locs:
    try:
        counties.append(geocoder.google(location).current_result.county)
    except:
        print("Issue querying: {}".format(location))
        
counties = Counter(counties)

county_names = list(counties.keys())
county_counts = list(counties.values())

county_fips = {x:0 for x in cali['FIPS'].tolist()}

for i in range(len(county_names)):
    try:
        fip = int(cali[cali['CTYNAME']==county_names[i]]['FIPS'])
        if fip:
            county_fips[fip] = county_counts[i]
    except:
        print("Issue with {}".format(county))

Issue querying: Menlo Park, CA


In [61]:
colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)'
]

fig = ff.create_choropleth(
    fips=list(county_fips.keys()), values=list(county_fips.values()), scope=['CA'], colorscale=colorscale,
#     binning_endpoints=[14348, 63983, 134827, 426762, 2081313], colorscale=colorscale,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5}, round_legend_values=True,
    legend_title='Population by County', title='California and Nearby States'
)
offline.iplot(fig, filename='choropleth_california_and_surr_states_outlines')